## Prompt tester

In [2]:
#r "nuget: Azure.AI.OpenAI, 1.0.0-beta.12"

Installed Packages Azure.AI.OpenAI, 1.0.0-beta.12

Setup

In [3]:
#!import ../config/Settings.cs

using Azure;
using Azure.AI.OpenAI;
using Microsoft.DotNet.Interactive;
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;

var (useAzureOpenAI, model, azureEndpoint, apiKey, bingApiKey, orgId) = Settings.LoadFromFile();

OpenAIClient client = new(new Uri(azureEndpoint), new AzureKeyCredential(apiKey));

Run various prompts

In [ ]:
do
{
    string? fileName = await InteractiveKernel.GetInputAsync("File name");
    if (string.IsNullOrEmpty(fileName))
        break;

    var messages = new ChatCompletionsOptions()
    {
        DeploymentName = model,
        Messages =
        {
            new ChatRequestSystemMessage(System.IO.File.ReadAllText($"{fileName}.txt"))
        },
        MaxTokens = 100
    };


    string? userMessage = await InteractiveKernel.GetInputAsync("User message:");
    messages.Messages.Add(new ChatRequestUserMessage(userMessage));
    using(StreamingResponse<StreamingChatCompletionsUpdate> response = await client.GetChatCompletionsStreamingAsync(messages))
    {
        var txt = new StringBuilder();
        await foreach (var choice in response)
        {
            if (choice.ContentUpdate != null)
            {
                var txt1 = choice.ContentUpdate;
                txt.Append(txt1);
            }
        }
        Console.Write(txt);
        Console.WriteLine();
    }
} while (true);